# Async

General note: Code shown here are for educational proposes only. In production, use a ready runtime like 

For IO operations, there is two way, blocking and non blocking. Here is a blocking sleep:

In [3]:
use std::{time::Duration, thread::sleep};

fn sleep_1s_and_return_5() -> i32 {
    sleep(Duration::from_secs(1));
    5
}

sleep_1s_and_return_5()

5

And here is a non blocking sleep. `Poll<T>` is a topologically equal enum to `Option<T>`. We like different types with equal topology in Rust, because it will prevent misusing a `Poll<T>` when an `Option<T>` is needed. A non blocking version is fairly more complex:

In [4]:
use std::{task::Poll, time::Instant};

struct Sleep {
    start_time: Instant,
}

impl Sleep {
    fn try_again(&self) -> Poll<i32> {
        if Instant::now() - self.start_time > Duration::from_secs(1) {
            Poll::Ready(5)
        } else {
            Poll::Pending
        }
    }
}

fn sleep_1s_and_return_5() -> Sleep {
    Sleep { start_time: Instant::now() }
}

let sleep_result = sleep_1s_and_return_5();
loop {
    match sleep_result.try_again() {
        Poll::Ready(x) => break x,
        Poll::Pending => println!("pending"),
    }
    sleep(Duration::from_millis(100)); // to prevent spamming the stdout
}

pending
pending
pending
pending
pending
pending
pending
pending
pending
pending


5

Trait `Future` abstracts the functionality we implemented above. Let's use that instead:

In [21]:
use std::{future::Future, task::{Context, Wake, Waker}, pin::Pin, sync::Arc};

struct Sleep {
    start_time: Instant,
}

impl Future for Sleep {
    type Output = i32;
    fn poll(self: Pin<&mut Sleep>, _: &mut Context) -> Poll<i32> {
        if Instant::now() - self.start_time > Duration::from_secs(1) {
            Poll::Ready(5)
        } else {
            Poll::Pending
        }
    }
}

struct EmptyWaker;
impl Wake for EmptyWaker {
    fn wake(self: Arc<Self>) {
    }
}

fn sleep_1s_and_return_5() -> impl Future<Output = i32> {
    Sleep { start_time: Instant::now() }
}
{
    let future = &mut sleep_1s_and_return_5();
    let mut sleep_result = Pin::new(future);
    let waker = Arc::new(EmptyWaker).into();
    let mut context = Context::from_waker(&waker);
    loop {
        match sleep_result.as_mut().poll(&mut context) {
            Poll::Ready(x) => break x,
            Poll::Pending => println!("pending"),
        }
        sleep(Duration::from_millis(100)); // to prevent spamming the stdout
    }
}

Error: mismatched types

Error: mismatched types

Too much ceremony added. Let's break them:
* `Future` is a trait, which represents tasks that can be polled.
* `Context` can provide a way for tasks to notify the runner they need to be polled. We didn't notify the runtime, so our implementation is technically wrong and runner is allowed to not poll our future, but since we provide runner as well, it works.

Now imagine we want to write a non blocking function which calls that function two times and print between calls. Blocking version is here:


In [36]:
fn sleep_1s_and_return_5() -> i32 {
    sleep(Duration::from_secs(1));
    5
}

fn composed_job() -> i32 {
    println!("start");
    let x = sleep_1s_and_return_5();
    println!("middle");
    let y = sleep_1s_and_return_5();
    println!("end");
    x + y
}

composed_job()

start
middle
end


10

Manual non blocking version would be way more complex:

In [25]:
fn sleep_1s_and_return_5() -> Sleep {
    Sleep { start_time: Instant::now() }
}

struct Sleep {
    start_time: Instant,
}

impl Future for Sleep {
    type Output = i32;
    fn poll(self: Pin<&mut Sleep>, _: &mut Context) -> Poll<i32> {
        if Instant::now() - self.start_time > Duration::from_secs(1) {
            Poll::Ready(5)
        } else {
            Poll::Pending
        }
    }
}

enum ComposedJobStateMachine {
    NotStarted,
    Beginning { fut1: Pin<Box<Sleep>> },
    Middle { x: i32, fut2: Pin<Box<Sleep>> },
    End { x: i32, y: i32 },
}

use ComposedJobStateMachine::*;

impl Future for ComposedJobStateMachine {
    type Output = i32;
    fn poll(mut self: Pin<&mut ComposedJobStateMachine>, cx: &mut Context) -> Poll<i32> {
        match &mut *self {
            NotStarted => {
                println!("begining");
                *self = Beginning {
                    fut1: Box::pin(sleep_1s_and_return_5()),
                };
                Poll::Pending
            }
            Beginning { fut1 } => {
                match fut1.as_mut().poll(cx) {
                    Poll::Ready(x) => {
                        println!("middle");
                        *self = Middle {
                            x, fut2: Box::pin(sleep_1s_and_return_5()),
                        };
                        Poll::Pending
                    }
                    Poll::Pending => Poll::Pending,
                }
            }
            Middle { x, fut2 } => {
                match fut2.as_mut().poll(cx) {
                    Poll::Ready(y) => {
                        println!("end");
                        let x = *x;
                        *self = End { x, y };
                        Poll::Ready(x + y)
                    }
                    Poll::Pending => Poll::Pending,
                }
            }
            End { x, y } => Poll::Ready(*x + *y),
        }
    }
}

fn composed_job() -> impl Future<Output = i32> {
    NotStarted
}

{
    let future = &mut composed_job();
    let mut sleep_result = Pin::new(future);
    let waker = Arc::new(EmptyWaker).into();
    let mut context = Context::from_waker(&waker);
    loop {
        match sleep_result.as_mut().poll(&mut context) {
            Poll::Ready(x) => break x,
            Poll::Pending => print!("pending "),
        }
        sleep(Duration::from_millis(100)); // to prevent spamming the stdout
    }
}

begining
pending pending pending pending pending pending pending pending pending pending middle
pending pending pending pending pending pending pending pending pending pending end


10

We created a state machine to keep which line of code we were in the last call. Since creating this stack machine is a very good way to compose multiple non blocking tasks, Rust supports it at the language level. We can write the above code with `async` blocks:

In [29]:
fn composed_job() -> impl Future<Output = i32> {
    async {
        println!("start");
        let x = sleep_1s_and_return_5().await;
        println!("middle");
        let y = sleep_1s_and_return_5().await;
        println!("end");
        x + y
    }
}

{
    let future = composed_job();
    let mut sleep_result = Box::pin(future);
    let waker = Arc::new(EmptyWaker).into();
    let mut context = Context::from_waker(&waker);
    loop {
        match sleep_result.as_mut().poll(&mut context) {
            Poll::Ready(x) => break x,
            Poll::Pending => print!("pending "),
        }
        sleep(Duration::from_millis(100)); // to prevent spamming the stdout
    }
}

start
pending pending pending pending pending pending pending pending pending pending middle
pending pending pending pending pending pending pending pending pending pending end


10

The `async` version looks like blocking, but works like non-blocking. Also Rust version is more performant. It doesn't have heap allocation and instead uses a self referential structure as state machine. `Pin`, which you have seen in above codes, is for this. `Pin<Pointer<T>>` is a type that guarantees that `T` won't be moved. `Pin` is very important for self referential data structures, since moving self referential data structres invalidates them.

Now that we have `async`, we can write concourent code which runs in a single thread:

In [ ]:
fn concurent_job() -> impl Future<Output = i32> {
    join!(
        async {
            println!("start");
            let x = sleep_1s_and_return_5().await;
            println!("middle");
            let y = sleep_1s_and_return_5().await;
            println!("end");
            x + y
        }
    )
}

{
    let future = concurent_job();
    let mut sleep_result = Box::pin(future);
    let waker = Arc::new(EmptyWaker).into();
    let mut context = Context::from_waker(&waker);
    loop {
        match sleep_result.as_mut().poll(&mut context) {
            Poll::Ready(x) => break x,
            Poll::Pending => print!("pending "),
        }
        sleep(Duration::from_millis(100)); // to prevent spamming the stdout
    }
}